In [1]:
import numpy as np
import cmath
import math
from scipy.optimize import least_squares
import time

# Gauss-Legendre 64-point nodes and weights (half-range)
x64 = np.array([
    0.0243502926634244325089558, 0.0729931217877990394495429, 0.1214628192961205544703765,
    0.1696444204239928180373136, 0.2174236437400070841496487, 0.2646871622087674163739642,
    0.3113228719902109561575127, 0.3572201583376681159504426, 0.4022701579639916036957668,
    0.4463660172534640879849477, 0.4894031457070529574785263, 0.5312794640198945456580139,
    0.5718956462026340342838781, 0.6111553551723932502488530, 0.6489654712546573398577612,
    0.6852363130542332425635584, 0.7198818501716108268489402, 0.7528199072605318966118638,
    0.7839723589433414076102205, 0.8132653151227975597419233, 0.8406292962525803627518615,
    0.8659993981540928197607834, 0.8893154459951141058534040, 0.9105221370785028057563807,
    0.9295691721319395758214902, 0.9464113748584028160624815, 0.9610087996520537189186141,
    0.9733268277899109637418535, 0.9833362538846259569312993, 0.9910133714767443207393824,
    0.9963401167719552793469245, 0.9993050417357721394569056
])
w64 = np.array([
    0.0486909570091397203833654, 0.0485754674415034269347991, 0.0483447622348029571697695,
    0.0479993885964583077281262, 0.0475401657148303086622822, 0.0469681828162100173253263,
    0.0462847965813144172959532, 0.0454916279274181444797710, 0.0445905581637565630601347,
    0.0435837245293234533768279, 0.0424735151236535890073398, 0.0412625632426235286101563,
    0.0399537411327203413866569, 0.0385501531786156291289625, 0.0370551285402400460404151,
    0.0354722132568823838106931, 0.0338051618371416093915655, 0.0320579283548515535854675,
    0.0302346570724024788679741, 0.0283396726142594832275113, 0.0263774697150546586716918,
    0.0243527025687108733381776, 0.0222701738083832541592983, 0.0201348231535302093723403,
    0.0179517157756973430850453, 0.0157260304760247193219660, 0.0134630478967186425980608,
    0.0111681394601311288185905, 0.0088467598263639477230309, 0.0065044579689783628561174,
    0.0041470332605624676352875, 0.0017832807216964329472961
])

# alias integration nodes/weights
glaw_u = x64
law_w = w64  # alias for consistency
glaw_w = w64

# Integration domain parameters
lb = 0.0
ub = 200.0
Q = 0.5 * (ub - lb)
P = 0.5 * (ub + lb)
pi = math.pi

# Market data container
def get_market_data():
    """
    Returns: spot S, rate r, strikes K, expiries T
    """
    K = np.array([
        0.9371, 0.8603, 0.8112, 0.7760, 0.7470, 0.7216, 0.6699, 0.6137,
        0.9956, 0.9868, 0.9728, 0.9588, 0.9464, 0.9358, 0.9175, 0.9025,
        1.0427, 1.0463, 1.0499, 1.0530, 1.0562, 1.0593, 1.0663, 1.0766,
        1.2287, 1.2399, 1.2485, 1.2659, 1.2646, 1.2715, 1.2859, 1.3046,
        1.3939, 1.4102, 1.4291, 1.4456, 1.4603, 1.4736, 1.5005, 1.5328
    ])
    T = np.array([
        0.119047619047619, 0.238095238095238, 0.357142857142857, 0.476190476190476,
        0.595238095238095, 0.714285714285714, 1.07142857142857, 1.42857142857143,
        0.119047619047619, 0.238095238095238, 0.357142857142857, 0.476190476190476,
        0.595238095238095, 0.714285714285714, 1.07142857142857, 1.42857142857143,
        0.119047619047619, 0.238095238095238, 0.357142857142857, 0.476190476190476,
        0.595238095238095, 0.714285714285714, 1.07142857142857, 1.42857142857143,
        0.119047619047619, 0.238095238095238, 0.357142857142857, 0.476190476190476,
        0.595238095238095, 0.714285714285714, 1.07142857142857, 1.42857142857143,
        0.119047619047619, 0.238095238095238, 0.357142857142857, 0.476190476190476,
        0.595238095238095, 0.714285714285714, 1.07142857142857, 1.42857142857143
    ])
    S = 1.0
    r = 0.02
    return S, r, K, T

# Integrand for option price

def HesIntMN(u, a, b, c, rho, v0, K, T, S, r):
    csqr = c * c
    PQ_M = P + Q * u
    PQ_N = P - Q * u
    imPQ_M = 1j * PQ_M
    imPQ_N = 1j * PQ_N
    _imPQ_M = 1j * (PQ_M - 1j)
    _imPQ_N = 1j * (PQ_N - 1j)

    h_M = K ** (-imPQ_M) / imPQ_M
    h_N = K ** (-imPQ_N) / imPQ_N

    x0 = math.log(S) + r * T
    tmp = c * rho
    kes_M1 = a - tmp * _imPQ_M
    kes_N1 = a - tmp * _imPQ_N
    kes_M2 = kes_M1 + tmp
    kes_N2 = kes_N1 + tmp

    m_M1 = imPQ_M + 1 + (PQ_M - 1j) ** 2
    m_N1 = imPQ_N + 1 + (PQ_N - 1j) ** 2
    m_M2 = imPQ_M + (PQ_M) ** 2
    m_N2 = imPQ_N + (PQ_N) ** 2

    d_M1 = cmath.sqrt(kes_M1 ** 2 + m_M1 * csqr)
    d_N1 = cmath.sqrt(kes_N1 ** 2 + m_N1 * csqr)
    d_M2 = cmath.sqrt(kes_M2 ** 2 + m_M2 * csqr)
    d_N2 = cmath.sqrt(kes_N2 ** 2 + m_N2 * csqr)

    tmp1 = -a * b * rho * T / c
    g_M2 = cmath.exp(tmp1 * imPQ_M)
    g_N2 = cmath.exp(tmp1 * imPQ_N)
    g_M1 = g_M2 * math.exp(tmp1)
    g_N1 = g_N2 * math.exp(tmp1)

    halft = 0.5 * T

    def ch_sinh_cosh(d):
        return cmath.cosh(d * halft), cmath.sinh(d * halft)

    calp_M1, salp_M1 = ch_sinh_cosh(d_M1)
    calp_N1, salp_N1 = ch_sinh_cosh(d_N1)
    calp_M2, salp_M2 = ch_sinh_cosh(d_M2)
    calp_N2, salp_N2 = ch_sinh_cosh(d_N2)

    A2_M1 = d_M1 * calp_M1 + kes_M1 * salp_M1
    A2_N1 = d_N1 * calp_N1 + kes_N1 * salp_N1
    A2_M2 = d_M2 * calp_M2 + kes_M2 * salp_M2
    A2_N2 = d_N2 * calp_N2 + kes_N2 * salp_N2

    A1_M1 = m_M1 * salp_M1
    A1_N1 = m_N1 * salp_N1
    A1_M2 = m_M2 * salp_M2
    A1_N2 = m_N2 * salp_N2

    A_M1 = A1_M1 / A2_M1
    A_N1 = A1_N1 / A2_N1
    A_M2 = A1_M2 / A2_M2
    A_N2 = A1_N2 / A2_N2

    tmp2 = 2 * a * b / (c * c)

    def D_factor(d, kes):
        return (cmath.log(d) + (a - d) * halft -
                cmath.log((d + kes) / 2 + (d - kes) / 2 * cmath.exp(-d * T)))

    D_M1 = D_factor(d_M1, kes_M1)
    D_M2 = D_factor(d_M2, kes_M2)
    D_N1 = D_factor(d_N1, kes_N1)
    D_N2 = D_factor(d_N2, kes_N2)

    y_M1 = cmath.exp(x0 * _imPQ_M - v0 * A_M1 + tmp2 * D_M1) * g_M1
    y_N1 = cmath.exp(x0 * _imPQ_N - v0 * A_N1 + tmp2 * D_N1) * g_N1
    y_M2 = cmath.exp(x0 * imPQ_M   - v0 * A_M2 + tmp2 * D_M2) * g_M2
    y_N2 = cmath.exp(x0 * imPQ_N   - v0 * A_N2 + tmp2 * D_N2) * g_N2

    M1 = (h_M * y_M1).real
    N1 = (h_N * y_N1).real
    M2 = (h_M * y_M2).real
    N2 = (h_N * y_N2).real

    return M1, N1, M2, N2

# Heston pricer: compute option prices for parameter set p
def fHes(p, S, r, K, T):
    a, b, c, rho, v0 = p
    prices = np.zeros_like(K)
    for idx in range(len(K)):
        k = K[idx]
        t = T[idx]
        disc = math.exp(-r * t)
        tmp = 0.5 * (S - k * disc)
        factor = disc / pi
        Y1 = 0.0
        Y2 = 0.0
        for uj, wj in zip(glaw_u, glaw_w):
            m1, n1, m2, n2 = HesIntMN(uj, a, b, c, rho, v0, k, t, S, r)
            Y1 += wj * (m1 + n1)
            Y2 += wj * (m2 + n2)
        prices[idx] = tmp + factor * (Q * Y1 - k * Q * Y2)
    return prices

# Residuals for calibration: market prices minus model
def residuals(p, S, r, K, T, market_prices):
    return fHes(p, S, r, K, T) - market_prices

if __name__ == '__main__':
    # True parameters (generate synthetic market data)
    p_true = np.array([3.0, 0.10, 0.25, -0.8, 0.08])
    S, r, K, T = get_market_data()
    market_prices = fHes(p_true, S, r, K, T)

    # Basic test: residuals at true parameters should be near zero
    res = residuals(p_true, S, r, K, T, market_prices)
    assert np.allclose(res, np.zeros_like(res), atol=1e-8), "Residuals not zero at true parameters"

    # Initial guess and calibration
    p0 = np.array([1.2, 0.2, 0.3, -0.6, 0.2])
    print('Initial guess:', p0)
    start = time.time()
    result = least_squares(
        residuals, p0,
        args=(S, r, K, T, market_prices),
        jac='2-point', method='lm',
        xtol=1e-10, ftol=1e-10, gtol=1e-10,
        max_nfev=100
    )
    end = time.time()

    print('Optimization success:', result.success)
    print('Estimated parameters:', result.x)
    print('True parameters:', p_true)
    print(f'Time elapsed: {end - start:.4f} seconds')


Initial guess: [ 1.2  0.2  0.3 -0.6  0.2]
Optimization success: True
Estimated parameters: [ 3.    0.1   0.25 -0.8   0.08]
True parameters: [ 3.    0.1   0.25 -0.8   0.08]
Time elapsed: 1.5494 seconds
